In [1]:
# Starting point

from PIL import Image
#import Image
import csv
import cv2
import numpy as np
from matplotlib import pyplot as plt
from keras.preprocessing.image import img_to_array, load_img
import random

#Lists for the main parameters that will be extracted from the csv files and images
lines = []

# Reads through the lines of the driving log, appending steering data above abolute values higher than 0.085, and only 30% of those below
# thus preventing car tendencies to go straight (recomendation from the forums)

with open('Data/driving_log.csv') as cvsfile:
    reader = csv.reader(cvsfile)
    for line in reader:
        if abs(float(line[3])) >= 0.08:
            lines.append(line)
        else:
            if random.random() >= 0.7:
                lines.append(line)

# arrays to store recorded data from the Sim tool
images = []
measurements = []

for line in lines:
    #Find the current steering measurement and stores it in the array
    measurement = float(line[3])
    # steering correctio parameter
    steering_correction = 0.35
    
    steering_center = measurement
    steering_left = measurement + steering_correction
    steering_right = measurement - steering_correction
    #Select image path, finds the image in the current line and stores it in the array
    source_path = line[0]
    filename = source_path.split("_2017")[-1]
    file_path = "data/IMG/"
    
    #storing image by camera
    image_center = load_img(file_path + "center_2017" + filename)
    image_left   = load_img(file_path + "left_2017" + filename)
    image_right  = load_img(file_path + "right_2017" + filename)
    
    #Extend images and measurements list
    images.extend([image_center, image_left, image_right])
    measurements.extend([steering_center, steering_left, steering_right])
    

# Lists for normal + inverted data
augmented_images = []
augmented_measurements = []

#Augmenting training data twice by including flipped images and inverted steering measurements
for image, measurement in zip(images, measurements):
    image_arr = np.asarray(image)
    augmented_images.append(image_arr)
    augmented_measurements.append(measurement)
    
    #Flipping images
    image_flip = np.fliplr(image_arr)
    augmented_images.append(image_flip)
    # inverted steering value
    augmented_measurements.append(measurement * (-1.0))

# Piece of code to store data from the 2nd track, recorded in a different folder
#THIS CODE WAS NOT USED IN THE FINAL TRAINING VERSION AND VIDEO RECORDING
"""
lines_2 = []

images_2 = []
measurements_2 = []

with open('Data_Track_2/driving_log.csv') as cvsfile:
    reader = csv.reader(cvsfile)
    for line in reader:
        lines_2.append(line)

for line in lines_2:
    #Find the current steering measurement and stores it in the array
    measurement = float(line[3])
    measurements_2.append(measurement)
    
    #Select image path, finds the image in the current line and stores it in the array
    source_path = line[0]
    filename = "c" + source_path.split("c")[-1]
    current_path = "Data_Track_2/IMG/" + filename
    image = cv2.imread(current_path)
    images_2.append(image)


#Augmenting training data twice by including flipped images and inverted steering measurements
for image, measurements in zip(images_2, measurements_2):
    augmented_images.append(image)
    augmented_measurements.append(measurement)
    augmented_images.append(cv2.flip(image, 1))
    augmented_measurements.append(measurement * (-1.0))
"""
# Converts the arrays into Numpy arrays as required by keras 
X_train = np.array(augmented_images)
y_train = np.array(augmented_measurements)

from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Cropping2D, Convolution2D, MaxPooling2D, Dropout

#Function define for image resizing
def resize_img(input):
    from keras.backend import tf as ktf
    return ktf.image.resize_images(input, (64, 64))

# Lenet architecure as used in the lectures
model = Sequential()
#Normalizing data
model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(160, 320, 3), output_shape=(160, 320, 3)))
#Cropping not needed pixel data
model.add(Cropping2D(cropping=((70,25),(0,0)), input_shape=(160, 320, 3)))
#Resize calling
model.add(Lambda(resize_img))

model.add(Convolution2D(6, 5, 5, activation = "relu"))
model.add(MaxPooling2D())
model.add(Convolution2D(6, 5, 5, activation = "relu"))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(120))
model.add(Dropout(0.5))
model.add(Dense(84))
model.add(Dropout(0.5))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split= 0.2, shuffle=True, nb_epoch=1)

model.save('model_v2.h5')

Using TensorFlow backend.


Train on 9624 samples, validate on 2406 samples
Epoch 1/1
9624/9624 [==============================] - 69s - loss: 0.0722 - val_loss: 0.1391
